In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import tree
from sklearn import preprocessing

titanic_dataset = 'docs/titanic2.csv'
breast_cancer_dataset = 'docs/breast_cancer_dataset.csv'

titanic = pd.read_csv(titanic_dataset, header=None, delimiter=',', names=['Pclass', 'sex', 'age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Initial', 'Age_band', 
'Family_size', 'Alone', 'Fare_cat', 'Deck', 'Title', 'Is_Married', 'Survived']) # se lee el csv y se indican el nombre de las columnas

atributos_titanic = titanic.loc[:, 'Pclass':'Is_Married'] # selección de las columnas de atributos
objetivo_titanic = titanic['Survived'] # selección de la columna objetivo

In [13]:
def metodo_evaluacion_robusta(dataset, atributos, objetivo, N_EXP, CV):

    codificador_atributos = preprocessing.OrdinalEncoder() # Codificador adecuado para los atributos
    codificador_atributos.fit(atributos) 
    atributos_codificados = codificador_atributos.transform(atributos)

    codificador_objetivo = preprocessing.LabelEncoder() # Codificador adecuado para el objetivo
    objetivo_codificado = codificador_objetivo.fit_transform(objetivo)
    
    atributos_entrenamiento, atributos_prueba, objetivo_entrenamiento, objetivo_prueba = model_selection.train_test_split(
    atributos_codificados, objetivo_codificado,  # Conjuntos de datos a dividir, usando los mismos índices para ambos
    random_state=12345,  # Valor de la semilla aleatoria, para que el muestreo sea reproducible, a pesar de ser aleatorio
    test_size=.20  # Tamaño del conjunto de prueba
    )  

    clasif_arbol_decision = tree.DecisionTreeClassifier() # creamos el clasificador
    clasif_arbol_decision.fit(X=atributos_entrenamiento, y=objetivo_entrenamiento)
    
    lista_promedios = []

    for i in range(N_EXP):
        print('Iteración: ',i)
        scores = model_selection.cross_val_score(estimator=clasif_arbol_decision, X=atributos_prueba, y=objetivo_prueba, cv=CV, scoring='balanced_accuracy')
        print('Score: ',scores)
        promedio = scores.mean()
        lista_promedios.append(promedio)
        print('Promedio: ',scores.mean())
    
    #Calculamos la media los promedios, que es lo que devolvera el metodo
    media = sum(lista_promedios)/len(lista_promedios)
    print('Media: ', media)
        
    return media
    

In [14]:
metodo_evaluacion_robusta(titanic, atributos_titanic, objetivo_titanic, 10, 10)

Iteración:  0
Score:  [0.88311688 0.83766234 0.67532468 0.72077922 0.6038961  0.7625
 0.85       0.5625     0.7        0.80714286]
Promedio:  0.7402922077922078
Iteración:  1
Score:  [0.81168831 0.76623377 0.67532468 0.72077922 0.46103896 0.65
 0.85       0.6125     0.6375     0.70714286]
Promedio:  0.6892207792207793
Iteración:  2
Score:  [0.85714286 0.72077922 0.67532468 0.72077922 0.64935065 0.7625
 0.8        0.5125     0.7        0.80714286]
Promedio:  0.720551948051948
Iteración:  3
Score:  [0.81168831 0.72077922 0.67532468 0.72077922 0.53246753 0.65
 0.8        0.6125     0.6375     0.75714286]
Promedio:  0.6918181818181818
Iteración:  4
Score:  [0.81168831 0.83766234 0.67532468 0.72077922 0.6038961  0.7625
 0.85       0.6125     0.6375     0.75714286]
Promedio:  0.7268993506493506
Iteración:  5
Score:  [0.88311688 0.72077922 0.67532468 0.67532468 0.53246753 0.7
 0.85       0.5625     0.6375     0.75714286]
Promedio:  0.6994155844155845
Iteración:  6
Score:  [0.81168831 0.720779

0.7115795454545455